In [13]:
import pandas as pd
import numpy as np
import altair as alt
from vega_datasets import data

In [14]:
def extract_michigan_expenditures(file_path):
    df = pd.read_excel(file_path)
    df_clean = df.copy()
    df_clean['payee_state'] = df_clean['payee_state'].astype(str).str.strip().str.upper()
    condition = (df_clean['payee_state'] == 'MI') | (df_clean['payee_state'] == 'DETROIT')
    mi_df = df_clean[condition].copy()
    mi_df.loc[mi_df['payee_state'] == 'DETROIT', 'payee_state'] = 'MI'
    
    return mi_df

file_path = "Datasets/Expenditures/2025_mi_cfr_expenditures1.xlsx"

results = extract_michigan_expenditures(file_path)

print(type(results))
print(f"data shape: {results.shape}")


<class 'pandas.core.frame.DataFrame'>
data shape: (28566, 37)


In [15]:
len(results)

28566

In [16]:
def clean_payee_zip(results):
    print(f"清洗前总行数: {len(results):,}")
    
    results_clean = results.copy()
    results_clean = results_clean[results_clean['payee_zip'].notna()].copy()
    print(f"删除空值后: {len(results_clean):,} 行")
    
    results_clean['payee_zip'] = results_clean['payee_zip'].astype(str).str.strip()
    results_clean = results_clean[results_clean['payee_zip'].str.upper() != 'NAN'].copy()
    results_clean['payee_zip'] = results_clean['payee_zip'].str.replace(r'\D', '', regex=True)
    results_clean = results_clean[results_clean['payee_zip'] != ''].copy()
    results_clean['payee_zip'] = results_clean['payee_zip'].apply(lambda x: x[:5] if len(x) >= 5 else x)
    results_clean = results_clean[results_clean['payee_zip'].str.len() == 5].copy()
    results_clean = results_clean[results_clean['payee_zip'].str.isdigit()].copy()
    
    results_clean['payee_zip'] = pd.to_numeric(results_clean['payee_zip'], errors='coerce')
    results_clean = results_clean[results_clean['payee_zip'].notna()].copy()
    results_clean['payee_zip'] = results_clean['payee_zip'].astype(int)
    
    results_clean = results_clean[(results_clean['payee_zip'] >= 48000) & (results_clean['payee_zip'] < 50000)].copy()
    
    print(f"最终数据: {len(results_clean):,} 行")
    if len(results_clean) > 0:
        print(f"\nZip Code 示例:")
        print(results_clean['payee_zip'].head(10))
    
    return results_clean

results_clean_zip = clean_payee_zip(results)

print("\n" + "="*60)
print("清洗后的数据信息:")
print("="*60)
print(f"数据形状: {results_clean_zip.shape}")

if len(results_clean_zip) > 0:
    print(f"Payee Zip 数据类型: {results_clean_zip['payee_zip'].dtype}")
    print(f"Payee Zip 唯一值数量: {results_clean_zip['payee_zip'].nunique():,}")
    print(f"Payee Zip 范围: {results_clean_zip['payee_zip'].min()} - {results_clean_zip['payee_zip'].max()}")
else:
    print("警告: 没有数据！")


清洗前总行数: 28,566
删除空值后: 27,874 行
最终数据: 27,708 行

Zip Code 示例:
5     48933
7     48116
12    48836
13    48933
20    48843
26    48116
27    48843
36    48933
37    48116
38    48933
Name: payee_zip, dtype: int64

清洗后的数据信息:
数据形状: (27708, 37)
Payee Zip 数据类型: int64
Payee Zip 唯一值数量: 743
Payee Zip 范围: 48001 - 49969


In [17]:
print(len(results_clean_zip))

27708


In [18]:
# Read the zip to county mapping file
zip_county_fips = pd.read_csv('zip_county_fips.csv')

# Convert zip column to integer
zip_county_fips['zip'] = zip_county_fips['zip'].astype(int)

# Merge to add FIPS code
results_clean_zip = results_clean_zip.merge(
    zip_county_fips[['zip', 'County FIPS code']], 
    left_on='payee_zip', 
    right_on='zip', 
    how='left'
)

# Drop extra column and rename
results_clean_zip = results_clean_zip.drop(columns=['zip'])
results_clean_zip = results_clean_zip.rename(columns={'County FIPS code': 'payee_fips'})

# Convert payee_fips to integer (Int64 can handle NaN)
results_clean_zip['payee_fips'] = results_clean_zip['payee_fips'].astype('Int64')

# print(f"\nRecords with FIPS code: {results_clean_zip['payee_fips'].notna().sum():,}")
# print(f"Records without FIPS code: {results_clean_zip['payee_fips'].isna().sum():,}")
print("\nSample data:")
# results_clean_zip



Sample data:


In [19]:

results_clean_zip = results_clean_zip[
    results_clean_zip['amount'].notna() & 
    (results_clean_zip['amount'].astype(str).str.strip() != '')
]


results_clean_zip['amount'] = results_clean_zip['amount'].astype(float)


print(f"清理后的数据行数: {len(results_clean_zip)}")
print(f"amount列数据类型: {results_clean_zip['amount'].dtype}")

清理后的数据行数: 27697
amount列数据类型: float64


In [20]:
results_clean_zip_copy = results_clean_zip.copy()

In [21]:
# 读取人口数据


population = pd.read_csv('county_pop_FIPS.csv')

# 确保fips列的数据类型正确
population['fips'] = population['fips'].astype(int)

# 去掉逗号，然后转换为数值
population['pop_2020'] = population['pop_2020'].astype(str).str.replace(',', '').str.replace(' ', '')
population['pop_2020'] = pd.to_numeric(population['pop_2020'], errors='coerce')

# 先删除payee_fips为空的行
results_clean_zip = results_clean_zip[results_clean_zip['payee_fips'].notna()]

# 然后转换为整数
results_clean_zip['payee_fips'] = results_clean_zip['payee_fips'].astype(int)

# 【先合并】人口数据到results_clean_zip
results_clean_zip = results_clean_zip.merge(
    population[['fips', 'pop_2020']], 
    left_on='payee_fips', 
    right_on='fips', 
    how='left'
)

# 【再计算】人均amount
results_clean_zip['amount_per_capita'] = results_clean_zip['amount'] / results_clean_zip['pop_2020']

# 删除辅助列（可选）
results_clean_zip = results_clean_zip.drop(columns=['fips'])

# 查看结果
print(f"合并后的数据行数: {len(results_clean_zip)}")
print(f"有人口数据的行数: {results_clean_zip['pop_2020'].notna().sum()}")
print(results_clean_zip[['payee_fips', 'amount', 'pop_2020', 'amount_per_capita']].head(10))


合并后的数据行数: 27615
有人口数据的行数: 27615
   payee_fips    amount  pop_2020  amount_per_capita
0       26065    100.00    290427           0.000344
1       26093   1160.00    196976           0.005889
2       26093    300.00    196976           0.001523
3       26065     46.56    290427           0.000160
4       26093    463.92    196976           0.002355
5       26093     74.67    196976           0.000379
6       26093  11242.76    196976           0.057077
7       26065    218.00    290427           0.000751
8       26093    851.51    196976           0.004323
9       26065    100.00    290427           0.000344


In [22]:
results_clean_zip.head()

,doc_seq_no,expenditure_type,gub_elec_type,expense_id,detail_id,doc_stmnt_year,doc_type_desc,com_legal_name,common_name_acronym,cfr_com_id,...,vend_state,vend_zip,fundraiser,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,payee_fips,pop_2020,amount_per_capita
0,25-591892,Direct Expenditures,,25-602874,0,2025,July CS,MIKE MURPHY FOR MI SENATE,MIKE MURPHY FOR MI SENATE,521903,...,,,,NaN,NaN,NaN,NaN,26065,290427,0.000344
1,25-591892,Direct Expenditures,,25-602892,0,2025,July CS,MIKE MURPHY FOR MI SENATE,MIKE MURPHY FOR MI SENATE,521903,...,,,,NaN,NaN,NaN,NaN,26093,196976,0.005889
2,25-591892,Direct Expenditures,,25-626895,0,2025,July CS,MIKE MURPHY FOR MI SENATE,MIKE MURPHY FOR MI SENATE,521903,...,,,,NaN,NaN,NaN,NaN,26093,196976,0.001523
3,25-591892,Direct Expenditures,,25-626897,0,2025,July CS,MIKE MURPHY FOR MI SENATE,MIKE MURPHY FOR MI SENATE,521903,...,,NaN,,NaN,NaN,NaN,NaN,26065,290427,0.000160
4,25-591892,Direct Expenditures,,25-626938,0,2025,July CS,MIKE MURPHY FOR MI SENATE,MIKE MURPHY FOR MI SENATE,521903,...,,,,NaN,NaN,NaN,NaN,26093,196976,0.002355


In [23]:

counties = alt.topo_feature(data.us_10m.url, 'counties')
click = alt.selection_point(fields=['id'])

# Filter out invalid values
county_amount = results_clean_zip.groupby('payee_fips')['amount_per_capita'].sum().reset_index()
county_amount.columns = ['id', 'total_amount']
county_amount = county_amount[county_amount['total_amount'] >= 0]  # Remove negative values
county_amount = county_amount.dropna(subset=['total_amount'])  # Remove NaN values

# Apply log transformation
county_amount['log_total_amount'] = np.log1p(county_amount['total_amount'])

# Create the choropleth chart
michigan_choropleth = alt.Chart(counties).mark_geoshape(
    stroke='black',
    strokeWidth=0.5
).transform_filter(
    (alt.datum.id >= 26000) & (alt.datum.id < 27000)
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(county_amount, 'id', ['log_total_amount', 'total_amount'])
).encode(
    color=alt.Color('log_total_amount:Q', 
                    scale=alt.Scale(scheme='yellowgreenblue'),
                    legend=alt.Legend(title='Log Total Amount', orient='right')),
    opacity=alt.condition(click, alt.value(1), alt.value(0.4)),
    tooltip=[
        alt.Tooltip('id:N', title='County FIPS'),
        alt.Tooltip('total_amount:Q', title='Total Amount', format='$,.2f'),
        alt.Tooltip('log_total_amount:Q', title='Log Total Amount', format='.2f')
    ]
).project(
    type='albersUsa'
).properties(
    width=600,
    height=500,
    title='Michigan Counties - Log Total Amount by County'
).add_params(
    click
)

michigan_choropleth


alt.Chart(...)

In [24]:
michigan_choropleth.save('michigan_choropleth.html')